# CoT con LangChain (Avanzado)

## Definición
CoT en LangChain explicita pasos de razonamiento y, combinado con salida estructurada, mejora auditabilidad y depuración.

## Cuándo usarlo
- Cuando necesitas consistencia de razonamiento en tareas ambiguas.
- Cuando quieres comparar zero-shot vs few-shot con evidencia.

## Cuándo NO usarlo
- Tareas triviales/determinísticas donde una función tradicional resuelve mejor.
- Contextos de latencia ultra-baja donde el costo de tokens importa demasiado.

## Flujo conceptual

```mermaid
graph TD
A[Context Packet] --> B[Zero-shot CoT]
A --> C[Few-shot CoT]
C --> D[Evaluar]
D --> E[Refinar]
E --> F[Seleccionar mejor salida]
```

In [ ]:
# Setup de notebook
from __future__ import annotations

import importlib.util
import inspect
import json
import sys
from pathlib import Path
from IPython.display import Markdown, display

# Add project root to path
project_root = Path.cwd()
while not (project_root / "pyproject.toml").exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

def load_module(path: Path, module_name: str):
    spec = importlib.util.spec_from_file_location(module_name, path)
    if spec is None or spec.loader is None:
        raise RuntimeError(f"Cannot load module: {path}")
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

ROOT = project_root
print('Repo root:', ROOT)

In [ ]:
script = ROOT / '03_langchain_prompting/COT_LangChain/Notebooks/01_cot_langchain_avanzado.py'
module = load_module(script, 'cot_langchain_nb')
run_fn = module.run_cot_langchain
print('Script:', script)
print('Firma:', inspect.signature(run_fn))

## Código fuente principal (visible)

In [ ]:
print(inspect.getsource(run_fn)[:6000])

## Ejemplo 1: caso base

In [ ]:
result_1 = run_fn(verbose=False)
print('Modelo:', result_1['__model'])
print('Context hash:', result_1['__context_hash'])
print('Evaluación final:')
print(json.dumps(result_1['selected_eval'], ensure_ascii=False, indent=2))
print('Salida final:')
print(json.dumps(result_1['selected'], ensure_ascii=False, indent=2))

In [ ]:
display(Markdown('## Flujo real usado en ejecución'))
display(Markdown('```mermaid\n' + result_1['__flow_mermaid'] + '\n```'))

## Ejemplo 2: coqueteo (asistente latino experto en enamorar)

In [ ]:
profile_coqueteo = {
  "tipo_persona": "asistente latino experto en enamorar con elegancia",
  "gustos": [
    "salsa romantica",
    "cafes bohemios",
    "poesia urbana",
    "paseos nocturnos"
  ],
  "estilo": "carismatico, coqueto, respetuoso, humor picante sutil",
  "contexto": "quiere iniciar una conversacion de coqueteo sin frases prefabricadas"
}
result_2 = run_fn(profile=profile_coqueteo, verbose=False)
print('Context hash coqueteo:', result_2['__context_hash'])
print('Evaluación final coqueteo:')
print(json.dumps(result_2['selected_eval'], ensure_ascii=False, indent=2))
print('Salida final coqueteo:')
print(json.dumps(result_2['selected'], ensure_ascii=False, indent=2))

## Errores típicos en producción
- Few-shot con ejemplos sesgados: bloquea creatividad y generalización.
- Context packet sin deduplicar: sube costo y degrada enfoque.
- Feedback loop sin umbral claro: loops caros y poca mejora real.
- Rúbrica débil: falsa sensación de calidad.

## Autocrítica
- CoT mejora explicabilidad, pero no garantiza verdad factual.
- El mayor retorno viene del context engineering, no del truco de prompt.